In [1]:
import os
import sys

sys.path.append('../')

from xgboost import XGBRegressor
import xgboost as xgb
import pandas as pd

from experiments.microblog_deep import *
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import config

/home/niyan/.local/lib/python3.4/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


..


In [14]:
config.features_to_use_mb = [
    'google_word_emb',
    'unigram',
    'bigram',
    # 'stemmed_polarity',
    # 'stemmed_aptitude',
    # 'polarity',
]

In [15]:
# loading data...
X = joblib.load(config.DUMPED_VECTOR_DIR + 'mb_sequences.pkl')
features, fmap = get_features('mb')
y = joblib.load(os.path.join(config.DATA_DIR, 'vectors_mb_new', 'mb_scores.pkl'))


---------------------------------------------
Loading google_word_emb from  /home/niyan/SemEval17-05-kar/data/vectors_mb_new/mb_google_word_emb.pkl
Shape =  (2494, 300), type = <class 'numpy.ndarray'>
---------------------------------------------

---------------------------------------------
Loading unigram from  /home/niyan/SemEval17-05-kar/data/vectors_mb_new/mb_unigram.pkl
Shape =  (2494, 2111), type = <class 'numpy.ndarray'>
---------------------------------------------

---------------------------------------------
Loading bigram from  /home/niyan/SemEval17-05-kar/data/vectors_mb_new/mb_bigram.pkl
Shape =  (2494, 6372), type = <class 'numpy.ndarray'>
---------------------------------------------


In [16]:
# split data
X_train, X_dev, X_test, Y_train, Y_dev, Y_test = X[:1693], X, X[1693:], y[:1693], y, y[1693:]
ft_train, ft_dev, ft_test = features[:1693], features, features[1693:]

In [17]:
def cos_scorer(y, y_pred):
    return cosine_similarity(np.reshape(y, (1, -1)), np.reshape(y_pred, (1, -1)))

In [18]:
p_grid = {'n_estimators': [160], 
          'max_depth': [5, 7],
          'min_child_weight': [1, 3],
          'learning_rate': [0.1, 0.3],
          'gamma': [0.0, 0.1, 1.0],
         }
xgb_model = XGBRegressor()
best_model = GridSearchCV(xgb_model, param_grid=p_grid, cv=4, scoring=make_scorer(cos_scorer))
_ = best_model.fit(ft_train, Y_train)

### Next
* print feature importance: by `xgb.get_score`

In [19]:
print("best param: ", best_model.best_params_)
print("best score: ", best_model.best_score_)
pd.DataFrame(best_model.cv_results_).sort_values(by=['mean_test_score'])

best param:  {'min_child_weight': 3, 'learning_rate': 0.1, 'max_depth': 5, 'gamma': 0.1, 'n_estimators': 160}
best score:  0.672474150107


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_gamma,param_learning_rate,param_max_depth,param_min_child_weight,param_n_estimators,params,...,split1_test_score,split1_train_score,split2_test_score,split2_train_score,split3_test_score,split3_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
23,13.185579,0.036564,0.577528,0.859640,1,0.3,7,3,160,"{'min_child_weight': 3, 'learning_rate': 0.3, ...",...,0.559650,0.848342,0.566439,0.854845,0.623686,0.864386,0.037477,0.000115,0.026769,0.008688
21,9.769036,0.036768,0.589555,0.838309,1,0.3,5,3,160,"{'min_child_weight': 3, 'learning_rate': 0.3, ...",...,0.575717,0.824834,0.598953,0.846407,0.645866,0.844386,0.020516,0.000181,0.039152,0.008435
22,13.286687,0.036664,0.592854,0.858007,1,0.3,7,1,160,"{'min_child_weight': 1, 'learning_rate': 0.3, ...",...,0.559650,0.848342,0.572247,0.859370,0.643164,0.860984,0.075582,0.000136,0.031891,0.005755
20,9.787696,0.036685,0.597496,0.843051,1,0.3,5,1,160,"{'min_child_weight': 1, 'learning_rate': 0.3, ...",...,0.575717,0.824834,0.599360,0.852981,0.645866,0.844386,0.020944,0.000075,0.030096,0.010961
18,13.273500,0.036715,0.614527,0.873923,1,0.1,7,1,160,"{'min_child_weight': 1, 'learning_rate': 0.1, ...",...,0.620744,0.874877,0.617434,0.871288,0.664935,0.877923,0.062225,0.000147,0.039116,0.002703
7,13.110678,0.036869,0.618564,0.969685,0,0.3,7,3,160,"{'min_child_weight': 3, 'learning_rate': 0.3, ...",...,0.600463,0.966948,0.602338,0.967444,0.662889,0.973102,0.005759,0.000131,0.025757,0.002580
19,13.225810,0.036671,0.619711,0.873953,1,0.1,7,3,160,"{'min_child_weight': 3, 'learning_rate': 0.1, ...",...,0.618357,0.876113,0.610762,0.868635,0.671873,0.879793,0.010957,0.000016,0.033720,0.004308
14,13.139956,0.036804,0.621631,0.957314,0.1,0.3,7,1,160,"{'min_child_weight': 1, 'learning_rate': 0.3, ...",...,0.611366,0.955415,0.597232,0.955766,0.669869,0.960164,0.067255,0.000105,0.028326,0.001903
17,9.762589,0.036804,0.626164,0.858961,1,0.1,5,3,160,"{'min_child_weight': 3, 'learning_rate': 0.1, ...",...,0.639019,0.857270,0.628794,0.859002,0.659875,0.859384,0.019123,0.000146,0.030500,0.001066
16,9.784181,0.036667,0.628751,0.860694,1,0.1,5,1,160,"{'min_child_weight': 1, 'learning_rate': 0.1, ...",...,0.639019,0.857270,0.629452,0.863022,0.659875,0.859384,0.011909,0.000091,0.026649,0.002482


In [10]:
params = {
    'min_child_weight': 1,
    'max_depth': 5,
    'eta': 0.1,
    'gamma': 0.1,
}
d_tr = xgb.DMatrix(ft_train, Y_train)
bst = xgb.train(params, dtrain=d_tr, num_boost_round=160)

In [12]:
fs_raw = bst.get_score(importance_type='gain')
fs = {}
for fname, v in fmap.items():
    fs[fname] = sum([fs_raw['f' + str(fn_raw)] for fn_raw in range(v[0], v[0] + v[1]) if ('f' + str(fn_raw)) in fs_raw])

fs

{'google_word_emb': 177.67434612890054,
 'rf_bigram': 0.31498580500000006,
 'rf_unigram': 12.102233190418472}

In [11]:
d_ts = xgb.DMatrix(ft_test)
Y_pred = bst.predict(d_ts)
score_ts = cos_scorer(Y_test, Y_pred)
print("cosine score on test data: ", score_ts[0][0])

cosine score on test data:  0.667172270974
